# Preprocess GDPX3's `median_features.csv`

- Turn it into AnnData for eventual downstream convenience
- Scale against controls
- Feature select with PyCytominer

In [2]:
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read in files

In [ ]:
metadata_file = '../data/ginkgo-datapoints_gdpx3_data-package/Ginkgo Datapoints_GDPx3_metadata.csv'
features_file = '../data/median_features.csv'
obs = (
    pd.read_csv(metadata_file)
      .set_index('sample_id')
)
X = (
    pd.read_csv(features_file)
      .drop('container_id', axis=1)
      .set_index('sample_id')
)

print(obs.shape)
print(X.shape)

# Make sure they share indices (sample_id)

In [6]:
idx = obs.index.intersection(X.index)
obs = obs.reindex(idx)
X = X.reindex(idx)

# Make AnnData

In [ ]:
from anndata import AnnData
adata = AnnData(X=X, obs=obs)
adata

# Scale with respect to container + % DMSO

In [ ]:
from sklearn.preprocessing import RobustScaler

def scale_per_group(
    adata,
    groupby_keys=['plate', 'percent_volume_dmso'],
    control_label='DMSO',
    source_layer=None,
    target_layer='scaled',
    strict=False
):
    """
    Scale features using RobustScaler fit on controls per group.
    
    Parameters:
        adata : AnnData
        groupby_keys : list of str
            Columns in adata.obs to group by.
        control_label : str
            Label in 'compound' column to use as control.
        source_layer : str or None
            Source layer to scale. If None, uses adata.X.
        target_layer : str
            Layer to write scaled data into.
        strict : bool
            If True, raise error on missing controls; else skip group.
    """
    X = adata.X if source_layer is None else adata.layers[source_layer]
    X = X.copy()
    obs = adata.obs.copy()

    scaled = np.full(X.shape, np.nan, dtype=np.float32)

    group_labels = obs[groupby_keys].astype(str).agg('|'.join, axis=1)

    for group in group_labels.unique():
        group_mask = group_labels == group
        control_mask = group_mask & (obs['compound'] == control_label)

        if np.sum(control_mask) < 2:
            if strict:
                raise ValueError(f"Insufficient controls in group: {group}")
            continue

        scaler = RobustScaler().fit(X[control_mask])
        scaled[group_mask] = scaler.transform(X[group_mask])

    adata.layers[target_layer] = scaled
    return adata

scale_per_group(adata, groupby_keys=['container_id', 'percent_volume_dmso'])

# Feature select

In [ ]:
from pycytominer import feature_select

def apply_feature_selection(
    adata,
    layer='scaled',
    operations=['variance_threshold', 'correlation_threshold'],
    copy=True
):
    """
    Applies pycytominer feature_select to scaled data and updates adata with only selected features.

    Parameters
    ----------
    adata : AnnData
        The AnnData object to filter.
    layer : str
        Which layer to use for feature selection (e.g., 'scaled').
    operations : list of str
        Feature selection operations to apply.
    copy : bool
        If True, returns a new AnnData object. If False, modifies in place.

    Returns
    -------
    AnnData
        The filtered AnnData object.
    """
    if copy:
        adata = adata.copy()

    print(f"🔍 Applying feature selection on layer '{layer}' using operations: {operations}")

    X = adata.to_df(layer=layer)
    feature_columns = X.columns.tolist()
    print(f"📊 Starting with {len(feature_columns):,} features")

    X_selected = feature_select(
        X,
        features=feature_columns,
        operation=operations
    )

    print(f"✅ Retained {X_selected.shape[1]:,} features after selection ({len(feature_columns) - X_selected.shape[1]:,} removed)")

    adata = adata[:, X_selected.columns]

    return adata

apply_feature_selection(adata=adata, copy=False)